# Exploratory Data Analysis - POI-VIIRS Integration

This notebook provides comprehensive exploratory data analysis of the integrated POI-VIIRS dataset, including statistical analysis, correlation studies, spatial patterns, and key insights.

## Objectives
1. Perform comprehensive statistical analysis
2. Analyze correlations between POI categories and luminosity
3. Study spatial patterns and autocorrelation
4. Evaluate clustering results
5. Investigate anomalous patterns
6. Generate actionable insights


In [ ]:
# Import required libraries
import sys
import os
sys.path.append('../src')

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pathlib import Path
import json

# Import custom modules
from analysis.exploratory_analysis import ExploratoryAnalyzer

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

## 1. Load Integrated Dataset

Let's load the integrated POI-VIIRS dataset created in the previous step.

In [ ]:
# Initialize the exploratory analyzer
analyzer = ExploratoryAnalyzer(data_dir="../data", output_dir="../results")

print("Exploratory analyzer initialized")
print(f"Data directory: {analyzer.data_dir}")
print(f"Output directory: {analyzer.output_dir}")

In [ ]:
# Load the integrated dataset
print("Loading integrated POI-VIIRS dataset...")
integrated_gdf = analyzer.load_integrated_data()

print(f"Dataset loaded: {len(integrated_gdf)} records")
print(f"Columns: {list(integrated_gdf.columns)}")
print(f"CRS: {integrated_gdf.crs}")

# Display basic info
print(f"\nDataset shape: {integrated_gdf.shape}")
print(f"Memory usage: {integrated_gdf.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB")

# Show first few records
display(integrated_gdf.head())

## 2. Basic Descriptive Statistics

Let's start with comprehensive descriptive statistics of our integrated dataset.

In [ ]:
# Generate basic descriptive statistics
print("Calculating basic descriptive statistics...")
desc_stats = analyzer.basic_descriptive_stats(integrated_gdf)

print("\nBASIC DESCRIPTIVE STATISTICS")
print("=" * 50)

print(f"\nDataset Overview:")
print(f"  Total POIs: {desc_stats['total_pois']:,}")
print(f"  Unique Categories: {desc_stats['unique_categories']}")

if 'luminosity_stats' in desc_stats:
    print(f"\nLuminosity Statistics:")
    lum_stats = desc_stats['luminosity_stats']
    print(f"  Count: {lum_stats['count']:.0f}")
    print(f"  Mean: {lum_stats['mean']:.3f}")
    print(f"  Std Dev: {lum_stats['std']:.3f}")
    print(f"  Min: {lum_stats['min']:.3f}")
    print(f"  25%: {lum_stats['25%']:.3f}")
    print(f"  50% (Median): {lum_stats['50%']:.3f}")
    print(f"  75%: {lum_stats['75%']:.3f}")
    print(f"  Max: {lum_stats['max']:.3f}")

print(f"\nCategory Distribution:")
total_pois = desc_stats['total_pois']
for category, count in desc_stats['category_distribution'].items():
    percentage = (count / total_pois) * 100
    print(f"  {category}: {count:,} ({percentage:.1f}%)")

if 'spatial_extent' in desc_stats:
    extent = desc_stats['spatial_extent']
    print(f"\nSpatial Extent:")
    print(f"  Longitude: {extent['min_longitude']:.6f} to {extent['max_longitude']:.6f}")
    print(f"  Latitude: {extent['min_latitude']:.6f} to {extent['max_latitude']:.6f}")
    print(f"  Approximate area: {extent['width_km']:.1f} × {extent['height_km']:.1f} km")

In [ ]:
# Create detailed descriptive visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Comprehensive Descriptive Statistics', fontsize=16, fontweight='bold')

# 1. Category distribution (pie chart)
ax1 = axes[0, 0]
category_counts = integrated_gdf['category_group'].value_counts()
colors = plt.cm.Set3(np.linspace(0, 1, len(category_counts)))
wedges, texts, autotexts = ax1.pie(category_counts.values, labels=category_counts.index, 
                                  autopct='%1.1f%%', colors=colors, startangle=90)
ax1.set_title('POI Category Distribution')

# 2. Luminosity distribution (histogram with KDE)
ax2 = axes[0, 1]
if 'viirs_luminosity' in integrated_gdf.columns:
    ax2.hist(integrated_gdf['viirs_luminosity'], bins=40, alpha=0.7, 
             color='skyblue', edgecolor='black', density=True, label='Histogram')
    # Add KDE curve
    from scipy.stats import gaussian_kde
    kde_data = integrated_gdf['viirs_luminosity'].dropna()
    if len(kde_data) > 1:
        kde = gaussian_kde(kde_data)
        x_range = np.linspace(kde_data.min(), kde_data.max(), 100)
        ax2.plot(x_range, kde(x_range), 'red', linewidth=2, label='KDE')
    ax2.set_xlabel('VIIRS Luminosity')
    ax2.set_ylabel('Density')
    ax2.set_title('Luminosity Distribution')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

# 3. Category vs Luminosity (boxplot)
ax3 = axes[0, 2]
if 'viirs_luminosity' in integrated_gdf.columns:
    box_data = [integrated_gdf[integrated_gdf['category_group'] == cat]['viirs_luminosity'].values 
                for cat in category_counts.index]
    bp = ax3.boxplot(box_data, labels=[cat[:10] + '...' if len(cat) > 10 else cat 
                                      for cat in category_counts.index], patch_artist=True)
    
    # Color the boxes
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    ax3.set_ylabel('VIIRS Luminosity')
    ax3.set_title('Luminosity by Category')
    ax3.tick_params(axis='x', rotation=45)
    ax3.grid(True, alpha=0.3)

# 4. Spatial distribution
ax4 = axes[1, 0]
if 'viirs_luminosity' in integrated_gdf.columns:
    scatter = ax4.scatter(integrated_gdf.geometry.x, integrated_gdf.geometry.y,
                         c=integrated_gdf['viirs_luminosity'], s=20, alpha=0.7,
                         cmap='viridis', edgecolors='white', linewidths=0.2)
    plt.colorbar(scatter, ax=ax4, label='VIIRS Luminosity')
else:
    ax4.scatter(integrated_gdf.geometry.x, integrated_gdf.geometry.y, s=20, alpha=0.7)
ax4.set_xlabel('Longitude')
ax4.set_ylabel('Latitude')
ax4.set_title('Spatial Distribution of POIs')
ax4.grid(True, alpha=0.3)

# 5. Cluster distribution (if available)
ax5 = axes[1, 1]
if 'cluster' in integrated_gdf.columns:
    cluster_counts = integrated_gdf['cluster'].value_counts().sort_index()
    colors_cluster = plt.cm.tab10(np.linspace(0, 1, len(cluster_counts)))
    bars = ax5.bar(cluster_counts.index, cluster_counts.values, color=colors_cluster)
    ax5.set_xlabel('Cluster ID')
    ax5.set_ylabel('Number of POIs')
    ax5.set_title('Cluster Size Distribution')
    ax5.set_xticks(cluster_counts.index)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax5.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{int(height)}', ha='center', va='bottom')
else:
    ax5.text(0.5, 0.5, 'No cluster data available', ha='center', va='center', 
             transform=ax5.transAxes, fontsize=12)
    ax5.set_title('Cluster Analysis')

# 6. Anomaly distribution (if available)
ax6 = axes[1, 2]
if 'anomaly_type' in integrated_gdf.columns:
    anomaly_counts = integrated_gdf['anomaly_type'].value_counts()
    colors_anomaly = ['#28a745', '#ffc107', '#dc3545', '#6c757d'][:len(anomaly_counts)]
    wedges, texts, autotexts = ax6.pie(anomaly_counts.values, labels=anomaly_counts.index,
                                      autopct='%1.1f%%', colors=colors_anomaly, startangle=90)
    ax6.set_title('Anomaly Type Distribution')
else:
    ax6.text(0.5, 0.5, 'No anomaly data available', ha='center', va='center',
             transform=ax6.transAxes, fontsize=12)
    ax6.set_title('Anomaly Analysis')

plt.tight_layout()
plt.show()

## 3. Correlation Analysis

Let's analyze the correlations between POI categories and luminosity patterns.

In [ ]:
# Perform correlation analysis
print("Performing POI-luminosity correlation analysis...")
correlation_results = analyzer.poi_luminosity_correlation_analysis(integrated_gdf)

print("\nCORRELATION ANALYSIS RESULTS")
print("=" * 50)

# Display category-luminosity correlations
if 'category_luminosity_correlations' in correlation_results:
    print("\nCategory-Luminosity Correlations:")
    correlations = correlation_results['category_luminosity_correlations']
    
    # Sort by absolute correlation value
    sorted_correlations = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)
    
    for category, correlation in sorted_correlations:
        strength = "Strong" if abs(correlation) > 0.5 else "Moderate" if abs(correlation) > 0.3 else "Weak"
        direction = "Positive" if correlation > 0 else "Negative"
        print(f"  {category}: {correlation:.3f} ({strength} {direction})")

# Display statistical significance tests
if 'correlation_tests' in correlation_results:
    print("\nStatistical Significance Tests (Mann-Whitney U):")
    print("Category vs Others - Luminosity Comparison")
    print("-" * 60)
    
    for category, test_results in correlation_results['correlation_tests'].items():
        significance = "***" if test_results['p_value'] < 0.001 else "**" if test_results['p_value'] < 0.01 else "*" if test_results['p_value'] < 0.05 else "ns"
        print(f"{category}:")
        print(f"  Mean luminosity: {test_results['mean_luminosity']:.3f}")
        print(f"  Median luminosity: {test_results['median_luminosity']:.3f}")
        print(f"  P-value: {test_results['p_value']:.4f} {significance}")
        print(f"  Significant difference: {test_results['significant']}")
        print()

In [ ]:
# Create correlation visualization
if 'viirs_luminosity' in integrated_gdf.columns:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('POI-Luminosity Correlation Analysis', fontsize=16, fontweight='bold')
    
    # 1. Correlation heatmap
    ax1 = axes[0, 0]
    # Create dummy variables for categories
    category_dummies = pd.get_dummies(integrated_gdf['category_group'])
    corr_data = pd.concat([category_dummies, integrated_gdf[['viirs_luminosity']]], axis=1)
    correlation_matrix = corr_data.corr()
    
    # Extract correlations with luminosity
    lum_correlations = correlation_matrix['viirs_luminosity'].drop('viirs_luminosity')
    
    # Create heatmap
    im = ax1.imshow([[corr] for corr in lum_correlations.values], 
                   cmap='RdBu', vmin=-1, vmax=1, aspect='auto')
    ax1.set_yticks(range(len(lum_correlations)))
    ax1.set_yticklabels(lum_correlations.index, fontsize=10)
    ax1.set_xticks([0])
    ax1.set_xticklabels(['VIIRS Luminosity'])
    ax1.set_title('Category-Luminosity Correlations')
    
    # Add correlation values
    for i, corr in enumerate(lum_correlations.values):
        ax1.text(0, i, f'{corr:.3f}', ha='center', va='center', 
                fontweight='bold', color='white' if abs(corr) > 0.5 else 'black')
    
    plt.colorbar(im, ax=ax1)
    
    # 2. Mean luminosity by category (bar plot)
    ax2 = axes[0, 1]
    mean_luminosity = integrated_gdf.groupby('category_group')['viirs_luminosity'].mean().sort_values(ascending=False)
    colors = plt.cm.viridis(np.linspace(0, 1, len(mean_luminosity)))
    bars = ax2.bar(range(len(mean_luminosity)), mean_luminosity.values, color=colors)
    ax2.set_xticks(range(len(mean_luminosity)))
    ax2.set_xticklabels(mean_luminosity.index, rotation=45, ha='right')
    ax2.set_ylabel('Mean VIIRS Luminosity')
    ax2.set_title('Mean Luminosity by Category')
    ax2.grid(True, alpha=0.3)
    
    # Add value labels
    for i, bar in enumerate(bars):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                f'{height:.2f}', ha='center', va='bottom')
    
    # 3. Violin plot - luminosity distribution by category
    ax3 = axes[1, 0]
    categories = integrated_gdf['category_group'].unique()
    violin_data = [integrated_gdf[integrated_gdf['category_group'] == cat]['viirs_luminosity'].values 
                   for cat in categories]
    
    parts = ax3.violinplot(violin_data, positions=range(len(categories)), 
                          showmeans=True, showmedians=True)
    
    # Color the violin plots
    colors = plt.cm.Set3(np.linspace(0, 1, len(categories)))
    for i, pc in enumerate(parts['bodies']):
        pc.set_facecolor(colors[i])
        pc.set_alpha(0.7)
    
    ax3.set_xticks(range(len(categories)))
    ax3.set_xticklabels([cat[:10] + '...' if len(cat) > 10 else cat for cat in categories], 
                       rotation=45, ha='right')
    ax3.set_ylabel('VIIRS Luminosity')
    ax3.set_title('Luminosity Distribution by Category')
    ax3.grid(True, alpha=0.3)
    
    # 4. Scatter plot - spatial correlation
    ax4 = axes[1, 1]
    scatter = ax4.scatter(integrated_gdf.geometry.x, integrated_gdf['viirs_luminosity'],
                         c=[plt.cm.Set3(i/len(categories)) for i, cat in enumerate(categories) 
                            for _ in range(len(integrated_gdf[integrated_gdf['category_group'] == cat]))],
                         s=30, alpha=0.7, edgecolors='white', linewidths=0.2)
    ax4.set_xlabel('Longitude')
    ax4.set_ylabel('VIIRS Luminosity')
    ax4.set_title('Spatial vs Luminosity Correlation')
    ax4.grid(True, alpha=0.3)
    
    # Add trend line
    z = np.polyfit(integrated_gdf.geometry.x, integrated_gdf['viirs_luminosity'], 1)
    p = np.poly1d(z)
    ax4.plot(integrated_gdf.geometry.x, p(integrated_gdf.geometry.x), "r--", alpha=0.8, linewidth=2)
    
    # Calculate correlation
    spatial_corr = integrated_gdf.geometry.x.corr(integrated_gdf['viirs_luminosity'])
    ax4.text(0.05, 0.95, f'Correlation: {spatial_corr:.3f}', 
             transform=ax4.transAxes, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

## 4. Spatial Autocorrelation Analysis

Let's examine spatial patterns and autocorrelation in the luminosity data.

In [ ]:
# Perform spatial autocorrelation analysis
print("Performing spatial autocorrelation analysis...")
spatial_results = analyzer.spatial_autocorrelation_analysis(integrated_gdf)

print("\nSPATIAL AUTOCORRELATION RESULTS")
print("=" * 50)

if 'morans_i' in spatial_results and spatial_results['morans_i'] is not None:
    morans_i = spatial_results['morans_i']
    print(f"\nMoran's I Statistic: {morans_i:.4f}")
    
    # Interpret Moran's I
    if morans_i > 0.1:
        interpretation = "Strong positive spatial autocorrelation"
        meaning = "Similar luminosity values tend to cluster together"
    elif morans_i > 0.05:
        interpretation = "Moderate positive spatial autocorrelation"
        meaning = "Some clustering of similar luminosity values"
    elif morans_i > -0.05:
        interpretation = "No significant spatial autocorrelation"
        meaning = "Luminosity values are randomly distributed"
    elif morans_i > -0.1:
        interpretation = "Moderate negative spatial autocorrelation"
        meaning = "Dissimilar luminosity values tend to be neighbors"
    else:
        interpretation = "Strong negative spatial autocorrelation"
        meaning = "Strong checkerboard pattern of luminosity values"
    
    print(f"Interpretation: {interpretation}")
    print(f"Meaning: {meaning}")
    
    if 'spatial_autocorrelation' in spatial_results:
        print(f"Pattern: {spatial_results['spatial_autocorrelation'].title()} autocorrelation")
    
    if 'autocorrelation_strength' in spatial_results:
        strength = spatial_results['autocorrelation_strength']
        strength_desc = "Strong" if strength > 0.3 else "Moderate" if strength > 0.1 else "Weak"
        print(f"Strength: {strength_desc} ({strength:.4f})")

else:
    print("\nSpatial autocorrelation analysis could not be completed.")
    print("This might be due to insufficient data or computational constraints.")

In [ ]:
# Visualize spatial patterns
if 'viirs_luminosity' in integrated_gdf.columns:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Spatial Pattern Analysis', fontsize=16, fontweight='bold')
    
    # 1. Spatial distribution with luminosity
    ax1 = axes[0, 0]
    scatter = ax1.scatter(integrated_gdf.geometry.x, integrated_gdf.geometry.y,
                         c=integrated_gdf['viirs_luminosity'], s=50, alpha=0.8,
                         cmap='YlOrRd', edgecolors='black', linewidths=0.3)
    plt.colorbar(scatter, ax=ax1, label='VIIRS Luminosity')
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    ax1.set_title('Spatial Distribution of Luminosity')
    ax1.grid(True, alpha=0.3)
    
    # 2. Luminosity hotspots (high values)
    ax2 = axes[0, 1]
    # Define hotspots as top 20% luminosity values
    hotspot_threshold = integrated_gdf['viirs_luminosity'].quantile(0.8)
    hotspots = integrated_gdf[integrated_gdf['viirs_luminosity'] >= hotspot_threshold]
    
    ax2.scatter(integrated_gdf.geometry.x, integrated_gdf.geometry.y, 
               c='lightgray', s=20, alpha=0.5, label='All POIs')
    ax2.scatter(hotspots.geometry.x, hotspots.geometry.y,
               c='red', s=60, alpha=0.8, label=f'Hotspots (>{hotspot_threshold:.1f})')
    ax2.set_xlabel('Longitude')
    ax2.set_ylabel('Latitude')
    ax2.set_title(f'Luminosity Hotspots ({len(hotspots)} POIs)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Spatial clustering by category
    ax3 = axes[1, 0]
    categories = integrated_gdf['category_group'].unique()
    colors = plt.cm.Set3(np.linspace(0, 1, len(categories)))
    
    for i, category in enumerate(categories):
        cat_data = integrated_gdf[integrated_gdf['category_group'] == category]
        ax3.scatter(cat_data.geometry.x, cat_data.geometry.y,
                   c=[colors[i]], s=40, alpha=0.7, label=category)
    
    ax3.set_xlabel('Longitude')
    ax3.set_ylabel('Latitude')
    ax3.set_title('Spatial Distribution by Category')
    ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax3.grid(True, alpha=0.3)
    
    # 4. Distance vs luminosity relationship
    ax4 = axes[1, 1]
    # Calculate distance from center
    center_x = integrated_gdf.geometry.x.mean()
    center_y = integrated_gdf.geometry.y.mean()
    
    distances = np.sqrt((integrated_gdf.geometry.x - center_x)**2 + 
                       (integrated_gdf.geometry.y - center_y)**2)
    
    scatter = ax4.scatter(distances, integrated_gdf['viirs_luminosity'],
                         c=integrated_gdf['viirs_luminosity'], s=30, alpha=0.7,
                         cmap='viridis', edgecolors='white', linewidths=0.2)
    ax4.set_xlabel('Distance from Center (degrees)')
    ax4.set_ylabel('VIIRS Luminosity')
    ax4.set_title('Distance vs Luminosity')
    ax4.grid(True, alpha=0.3)
    
    # Add trend line
    z = np.polyfit(distances, integrated_gdf['viirs_luminosity'], 1)
    p = np.poly1d(z)
    ax4.plot(distances, p(distances), "r--", alpha=0.8, linewidth=2)
    
    # Calculate correlation
    dist_corr = distances.corr(integrated_gdf['viirs_luminosity'])
    ax4.text(0.05, 0.95, f'Correlation: {dist_corr:.3f}', 
             transform=ax4.transAxes, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # Print spatial pattern insights
    print(f"\nSpatial Pattern Insights:")
    print(f"  - Identified {len(hotspots)} luminosity hotspots (top 20%)")
    print(f"  - Hotspot threshold: {hotspot_threshold:.2f} luminosity units")
    print(f"  - Distance-luminosity correlation: {dist_corr:.3f}")
    
    if abs(dist_corr) > 0.3:
        pattern = "decreases" if dist_corr < 0 else "increases"
        print(f"  - Luminosity {pattern} with distance from center (moderate to strong correlation)")
    else:
        print(f"  - No strong distance-luminosity relationship detected")

## 5. Cluster Analysis Evaluation

Let's evaluate the quality and characteristics of our clustering results.

In [ ]:
# Perform cluster analysis
print("Evaluating cluster analysis results...")
cluster_results = analyzer.cluster_analysis(integrated_gdf)

print("\nCLUSTER ANALYSIS RESULTS")
print("=" * 50)

if 'cluster_summary' in cluster_results:
    print("\nCluster Summary Statistics:")
    print("-" * 30)
    
    cluster_summary = cluster_results['cluster_summary']
    for cluster_id in sorted(cluster_summary.keys()):
        cluster_stats = cluster_summary[cluster_id]
        print(f"\nCluster {cluster_id}:")
        
        if 'viirs_luminosity' in cluster_stats:
            lum_stats = cluster_stats['viirs_luminosity']
            print(f"  POI Count: {lum_stats['count']}")
            print(f"  Mean Luminosity: {lum_stats['mean']:.3f}")
            print(f"  Std Luminosity: {lum_stats['std']:.3f}")
            print(f"  Median Luminosity: {lum_stats['median']:.3f}")
        
        if 'category_group' in cluster_stats:
            dominant_category = cluster_stats['category_group']
            print(f"  Dominant Category: {dominant_category}")

# Silhouette score evaluation
if 'silhouette_score' in cluster_results:
    silhouette_score = cluster_results['silhouette_score']
    cluster_quality = cluster_results.get('cluster_quality', 'unknown')
    
    print(f"\nCluster Quality Assessment:")
    print(f"  Silhouette Score: {silhouette_score:.4f}")
    print(f"  Quality Rating: {cluster_quality.title()}")
    
    # Interpretation
    if silhouette_score > 0.7:
        interpretation = "Excellent clustering - clusters are well-separated and cohesive"
    elif silhouette_score > 0.5:
        interpretation = "Good clustering - reasonable structure with some overlap"
    elif silhouette_score > 0.3:
        interpretation = "Moderate clustering - clusters exist but with significant overlap"
    elif silhouette_score > 0:
        interpretation = "Weak clustering - minimal cluster structure"
    else:
        interpretation = "Poor clustering - data points may be better assigned to other clusters"
    
    print(f"  Interpretation: {interpretation}")

else:
    print("\nCluster quality assessment could not be performed.")

In [ ]:
# Visualize cluster analysis
if 'cluster' in integrated_gdf.columns:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Cluster Analysis Evaluation', fontsize=16, fontweight='bold')
    
    n_clusters = integrated_gdf['cluster'].nunique()
    cluster_colors = plt.cm.tab10(np.linspace(0, 1, n_clusters))
    
    # 1. Spatial distribution of clusters
    ax1 = axes[0, 0]
    for i, cluster_id in enumerate(sorted(integrated_gdf['cluster'].unique())):
        cluster_data = integrated_gdf[integrated_gdf['cluster'] == cluster_id]
        ax1.scatter(cluster_data.geometry.x, cluster_data.geometry.y,
                   c=[cluster_colors[i]], s=50, alpha=0.7, 
                   label=f'Cluster {cluster_id} ({len(cluster_data)} POIs)',
                   edgecolors='white', linewidths=0.5)
    
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    ax1.set_title('Spatial Distribution of Clusters')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # 2. Cluster characteristics (if luminosity available)
    ax2 = axes[0, 1]
    if 'viirs_luminosity' in integrated_gdf.columns:
        cluster_means = integrated_gdf.groupby('cluster')['viirs_luminosity'].mean()
        cluster_stds = integrated_gdf.groupby('cluster')['viirs_luminosity'].std()
        
        bars = ax2.bar(cluster_means.index, cluster_means.values, 
                      yerr=cluster_stds.values, capsize=5,
                      color=cluster_colors, alpha=0.7)
        ax2.set_xlabel('Cluster ID')
        ax2.set_ylabel('Mean VIIRS Luminosity')
        ax2.set_title('Mean Luminosity by Cluster')
        ax2.set_xticks(cluster_means.index)
        ax2.grid(True, alpha=0.3)
        
        # Add value labels
        for i, bar in enumerate(bars):
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height + cluster_stds.iloc[i] + 0.1,
                    f'{height:.2f}', ha='center', va='bottom')
    
    # 3. Cluster size distribution
    ax3 = axes[1, 0]
    cluster_sizes = integrated_gdf['cluster'].value_counts().sort_index()
    bars = ax3.bar(cluster_sizes.index, cluster_sizes.values, 
                   color=cluster_colors, alpha=0.7)
    ax3.set_xlabel('Cluster ID')
    ax3.set_ylabel('Number of POIs')
    ax3.set_title('Cluster Size Distribution')
    ax3.set_xticks(cluster_sizes.index)
    ax3.grid(True, alpha=0.3)
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{int(height)}', ha='center', va='bottom')
    
    # 4. Cluster composition by category
    ax4 = axes[1, 1]
    cluster_category_crosstab = pd.crosstab(integrated_gdf['cluster'], 
                                           integrated_gdf['category_group'], 
                                           normalize='index') * 100
    
    cluster_category_crosstab.plot(kind='bar', ax=ax4, stacked=True, 
                                  colormap='Set3', alpha=0.8)
    ax4.set_xlabel('Cluster ID')
    ax4.set_ylabel('Percentage of POIs')
    ax4.set_title('Cluster Composition by Category')
    ax4.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax4.set_xticklabels(ax4.get_xticklabels(), rotation=0)
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print cluster insights
    print(f"\nCluster Analysis Insights:")
    print(f"  - Total clusters identified: {n_clusters}")
    print(f"  - Cluster sizes range from {cluster_sizes.min()} to {cluster_sizes.max()} POIs")
    
    if 'viirs_luminosity' in integrated_gdf.columns:
        highest_lum_cluster = cluster_means.idxmax()
        lowest_lum_cluster = cluster_means.idxmin()
        print(f"  - Highest luminosity cluster: {highest_lum_cluster} (mean: {cluster_means[highest_lum_cluster]:.2f})")
        print(f"  - Lowest luminosity cluster: {lowest_lum_cluster} (mean: {cluster_means[lowest_lum_cluster]:.2f})")
    
    # Find most homogeneous cluster (by category)
    cluster_category_max = cluster_category_crosstab.max(axis=1)
    most_homogeneous_cluster = cluster_category_max.idxmax()
    homogeneity_score = cluster_category_max[most_homogeneous_cluster]
    print(f"  - Most homogeneous cluster: {most_homogeneous_cluster} ({homogeneity_score:.1f}% single category)")

else:
    print("\nNo cluster data available for visualization.")

## 6. Anomaly Analysis

Let's examine the anomalous patterns we've identified in detail.

In [ ]:
# Perform anomaly analysis
print("Analyzing anomalous POI-luminosity patterns...")
anomaly_results = analyzer.anomaly_analysis(integrated_gdf)

print("\nANOMALY ANALYSIS RESULTS")
print("=" * 50)

if 'anomaly_distribution' in anomaly_results:
    print("\nAnomaly Type Distribution:")
    anomaly_dist = anomaly_results['anomaly_distribution']
    total_pois = sum(anomaly_dist.values())
    
    for anomaly_type, count in anomaly_dist.items():
        percentage = (count / total_pois) * 100
        print(f"  {anomaly_type}: {count:,} ({percentage:.1f}%)")

if 'anomaly_characteristics' in anomaly_results:
    print("\nAnomaly Characteristics:")
    print("-" * 30)
    
    anomaly_chars = anomaly_results['anomaly_characteristics']
    for anomaly_type, characteristics in anomaly_chars.items():
        print(f"\n{anomaly_type}:")
        print(f"  Count: {characteristics['count']:,}")
        
        if 'mean_luminosity' in characteristics:
            print(f"  Mean Luminosity: {characteristics['mean_luminosity']:.3f}")
            print(f"  Median Luminosity: {characteristics['median_luminosity']:.3f}")
        
        if 'dominant_category' in characteristics and characteristics['dominant_category']:
            print(f"  Dominant Category: {characteristics['dominant_category']}")

# Provide interpretations
print("\nAnomaly Interpretations:")
print("-" * 25)

if 'anomaly_distribution' in anomaly_results:
    for anomaly_type in anomaly_results['anomaly_distribution'].keys():
        if anomaly_type == 'Normal':
            print(f"\n{anomaly_type}: POIs with expected luminosity levels for their category")
        elif anomaly_type == 'High Light, Non-Commercial':
            print(f"\n{anomaly_type}: Areas with high luminosity but non-commercial POIs")
            print(f"  → Possible interpretations:")
            print(f"    • Residential areas with good lighting infrastructure")
            print(f"    • Essential services in well-lit areas")
            print(f"    • Mixed-use areas with bright street lighting")
        elif anomaly_type == 'Low Light, Commercial':
            print(f"\n{anomaly_type}: Commercial POIs in areas with low luminosity")
            print(f"  → Possible interpretations:")
            print(f"    • Underlit commercial areas needing infrastructure improvement")
            print(f"    • Small-scale or informal commercial activities")
            print(f"    • Areas with potential safety or accessibility concerns")

## 7. Generate Comprehensive Analysis Report

Finally, let's generate a comprehensive report with all our findings.

In [ ]:
# Generate comprehensive analysis report
print("Generating comprehensive exploratory analysis report...")
comprehensive_report = analyzer.generate_analysis_report(integrated_gdf)

print("\n" + "=" * 70)
print("COMPREHENSIVE EXPLORATORY ANALYSIS COMPLETED")
print("=" * 70)

# Display key findings summary
print("\nKEY FINDINGS SUMMARY:")
print("-" * 30)

# Basic statistics
if 'descriptive_stats' in comprehensive_report:
    stats = comprehensive_report['descriptive_stats']
    print(f"\n📊 Dataset Overview:")
    print(f"   • Total POIs analyzed: {stats['total_pois']:,}")
    print(f"   • Categories represented: {stats['unique_categories']}")
    
    if 'luminosity_stats' in stats:
        lum_stats = stats['luminosity_stats']
        print(f"   • Luminosity range: {lum_stats['min']:.2f} to {lum_stats['max']:.2f}")
        print(f"   • Mean luminosity: {lum_stats['mean']:.2f}")

# Correlation findings
if 'correlation_analysis' in comprehensive_report:
    corr_analysis = comprehensive_report['correlation_analysis']
    if 'category_luminosity_correlations' in corr_analysis:
        correlations = corr_analysis['category_luminosity_correlations']
        print(f"\n🔗 Correlation Findings:")
        
        # Find strongest correlations
        sorted_corr = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)
        
        if len(sorted_corr) > 0:
            strongest_cat, strongest_corr = sorted_corr[0]
            print(f"   • Strongest correlation: {strongest_cat} ({strongest_corr:.3f})")
        
        # Identify significant correlations
        significant_corr = [(cat, corr) for cat, corr in correlations.items() if abs(corr) > 0.3]
        print(f"   • Categories with significant correlations: {len(significant_corr)}")

# Spatial pattern insights
if 'spatial_analysis' in comprehensive_report:
    spatial_analysis = comprehensive_report['spatial_analysis']
    if 'morans_i' in spatial_analysis and spatial_analysis['morans_i'] is not None:
        morans_i = spatial_analysis['morans_i']
        print(f"\n🗺️ Spatial Pattern Insights:")
        print(f"   • Moran's I statistic: {morans_i:.4f}")
        print(f"   • Spatial pattern: {spatial_analysis.get('spatial_autocorrelation', 'Unknown').title()}")
        
        if abs(morans_i) > 0.1:
            strength = "Strong" if abs(morans_i) > 0.3 else "Moderate"
            direction = "clustering" if morans_i > 0 else "dispersion"
            print(f"   • Pattern strength: {strength} {direction}")

# Cluster quality
if 'cluster_analysis' in comprehensive_report:
    cluster_analysis = comprehensive_report['cluster_analysis']
    if 'silhouette_score' in cluster_analysis:
        silhouette = cluster_analysis['silhouette_score']
        quality = cluster_analysis['cluster_quality']
        print(f"\n🎯 Clustering Results:")
        print(f"   • Cluster quality: {quality.title()}")
        print(f"   • Silhouette score: {silhouette:.3f}")

# Anomaly insights
if 'anomaly_analysis' in comprehensive_report:
    anomaly_analysis = comprehensive_report['anomaly_analysis']
    if 'anomaly_distribution' in anomaly_analysis:
        anomalies = anomaly_analysis['anomaly_distribution']
        print(f"\n⚠️ Anomaly Detection:")
        
        total_anomalies = sum([count for atype, count in anomalies.items() if atype != 'Normal'])
        total_pois = sum(anomalies.values())
        anomaly_rate = (total_anomalies / total_pois) * 100
        
        print(f"   • Anomalous POIs: {total_anomalies:,} ({anomaly_rate:.1f}%)")
        
        # Highlight specific anomaly types
        for atype, count in anomalies.items():
            if atype != 'Normal' and count > 0:
                percentage = (count / total_pois) * 100
                print(f"   • {atype}: {count} ({percentage:.1f}%)")

print("\n" + "=" * 70)
print("ANALYSIS OUTPUTS:")
print(f"📁 Results directory: {analyzer.output_dir}")
print(f"📊 Analysis report: {analyzer.output_dir}/exploratory_analysis_report.json")
print(f"📈 Summary plots: {analyzer.output_dir}/plots/exploratory_analysis_summary.png")
print("\nNext steps:")
print("1. Review detailed analysis report (JSON file)")
print("2. Run notebook 04_visualization.ipynb for interactive visualizations")
print("3. Use findings to inform urban planning decisions")
print("=" * 70)

In [ ]:
# Create a final comprehensive summary table
summary_data = {
    'Metric': [],
    'Value': [],
    'Interpretation': []
}

# Add key metrics
if 'descriptive_stats' in comprehensive_report:
    stats = comprehensive_report['descriptive_stats']
    
    summary_data['Metric'].extend([
        'Total POIs',
        'Categories',
        'Mean Luminosity',
        'Study Area'
    ])
    
    summary_data['Value'].extend([
        f"{stats['total_pois']:,}",
        str(stats['unique_categories']),
        f"{stats.get('luminosity_stats', {}).get('mean', 0):.2f}",
        f"{stats.get('spatial_extent', {}).get('width_km', 0):.1f} × {stats.get('spatial_extent', {}).get('height_km', 0):.1f} km"
    ])
    
    summary_data['Interpretation'].extend([
        'Comprehensive POI coverage',
        'Diverse urban functions represented',
        'Average nighttime illumination level',
        'Focused neighborhood-scale analysis'
    ])

# Add correlation insights
if 'correlation_analysis' in comprehensive_report:
    corr_analysis = comprehensive_report['correlation_analysis']
    if 'category_luminosity_correlations' in corr_analysis:
        correlations = corr_analysis['category_luminosity_correlations']
        strongest_corr = max(correlations.items(), key=lambda x: abs(x[1]))
        
        summary_data['Metric'].append('Strongest Category Correlation')
        summary_data['Value'].append(f"{strongest_corr[0]}: {strongest_corr[1]:.3f}")
        
        if abs(strongest_corr[1]) > 0.5:
            interpretation = "Strong luminosity association"
        elif abs(strongest_corr[1]) > 0.3:
            interpretation = "Moderate luminosity association"
        else:
            interpretation = "Weak luminosity association"
        
        summary_data['Interpretation'].append(interpretation)

# Add clustering results
if 'cluster_analysis' in comprehensive_report and 'silhouette_score' in comprehensive_report['cluster_analysis']:
    silhouette = comprehensive_report['cluster_analysis']['silhouette_score']
    quality = comprehensive_report['cluster_analysis']['cluster_quality']
    
    summary_data['Metric'].append('Clustering Quality')
    summary_data['Value'].append(f"{quality.title()} ({silhouette:.3f})")
    summary_data['Interpretation'].append('Spatial clustering effectiveness')

# Add anomaly rate
if 'anomaly_analysis' in comprehensive_report and 'anomaly_distribution' in comprehensive_report['anomaly_analysis']:
    anomalies = comprehensive_report['anomaly_analysis']['anomaly_distribution']
    total_anomalies = sum([count for atype, count in anomalies.items() if atype != 'Normal'])
    total_pois = sum(anomalies.values())
    anomaly_rate = (total_anomalies / total_pois) * 100
    
    summary_data['Metric'].append('Anomaly Rate')
    summary_data['Value'].append(f"{anomaly_rate:.1f}%")
    
    if anomaly_rate > 20:
        interpretation = "High anomaly rate - diverse patterns"
    elif anomaly_rate > 10:
        interpretation = "Moderate anomaly rate - some outliers"
    else:
        interpretation = "Low anomaly rate - consistent patterns"
    
    summary_data['Interpretation'].append(interpretation)

# Create and display summary table
summary_df = pd.DataFrame(summary_data)

print("\nFINAL ANALYSIS SUMMARY TABLE")
print("=" * 80)
display(summary_df)

# Save summary table
summary_df.to_csv(analyzer.output_dir / 'exploratory_analysis_summary.csv', index=False)
print(f"\nSummary table saved to: {analyzer.output_dir}/exploratory_analysis_summary.csv")

print("\n🎉 EXPLORATORY DATA ANALYSIS COMPLETE! 🎉")